Installing pdfplumber

In [ ]:
pip install pdfplumber

Installing fuzzywuzzy

In [ ]:
pip install fuzzywuzzy

Imports

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import re
import os
import datetime
import pdfplumber
from fuzzywuzzy import fuzz

User-defined Functions and Dictionaries

In [ ]:
# Matching Functions

def match_or_not_vsp(fullname_day, fullname_ins, date_day, date_ins):

    if date_day != date_ins:
        return 0
    else:    
        name_parts_re = re.compile(r'([\.\'A-Z-]+)\s+([\.\'A-Z-]+)')
    
        firstname_day = name_parts_re.search(fullname_day).group(1).strip()
        lastname_day = name_parts_re.search(fullname_day).group(2).strip()

        firstname_ins = name_parts_re.search(fullname_ins).group(1).strip()
        lastname_ins = name_parts_re.search(fullname_ins).group(2).strip()

        if firstname_day[:2] == firstname_ins[:2]:
            if round(len(lastname_day) - ((fuzz.partial_ratio(lastname_day, lastname_ins) / 100) * len(lastname_day))) <= 1 or round(len(lastname_ins) - ((fuzz.partial_ratio(lastname_day, lastname_ins) / 100) * len(lastname_ins))) <= 1:
                return 1
            else:
                return 0
        else:
            return 0





Regular Expressions

In [ ]:
# EyeMed (website)
eyemed_claim_num_re = re.compile(r'Claim #: (\d{12})')
eyemed_name_re = re.compile(r'Member Name: ([A-Z- \'\.]+), ([A-Z- \'\.]+) Subscriber #:')
eyemed_dos_re = re.compile(r'(\d{2}/\d{2}/\d{2}) \d{5}')
eyemed_payments_re = re.compile(r'Totals -?\$(\d{1,3}\.\d{2}) -?\$(\d{1,3}\.\d{2}) -?\$(\d{1,3}\.\d{2}) -?\$(\d{1,3}\.\d{2}) -?\$(\d{1,3}\.\d{2}) -?\$(\d{1,3}\.\d{2}) -?\$(\d{1,3}\.\d{2}) -?\$(\d{1,3}\.\d{2})')
eyemed_rename_re = re.compile(r'(\d{2}/\d{2}/\d{2})')
# EyeMed (scanned)
scanned_eyemed_name_claim_num_re = re.compile(r'(\d{11}) +(.+), +(.+)')
scanned_eyemed_dos_re = re.compile(r'(\d{1,2}/\d{2}/\d{4}) +\d{4,5}')
scanned_eyemed_payments_re = re.compile(r'Total Auth: +\d{10} +\d{0,3}[,\.\' ]*\d{2} +\d{0,3}[,\.\' ]*\d{2} +\d{0,3}[,\.\' ]*\d{2} +\d{0,3}[,\.\' ]*\d{2} +\d{0,3}[,\.\' ]*\d{2} +\d{0,3}[,\.\' ]*\d{2} +\d{0,3}[,\.\' ]*\d{2} +(\d{1,3})[,\.\' ]*\d{2}')
scanned_eyemed_rename_re = re.compile(r'Date: (\d{1,2}/\d{2}/\d{4})')
# VSP
vsp_name_claim_num_re = re.compile(r'[ \d]([-A-Z\'\.]+), ([A-Z-\'\.]*) (\d{10})')
vsp_dos_re = re.compile(r'(\d{1,2}/\d{2}/\d{2}) +(\d{5})')
vsp_payments_re = re.compile(r'Totals -?\d{1,3}\.\d{2} -?\d{1,2}\.\d{2} -?\d{1,2}\.\d{2} -?\d{1,2}\.\d{2} -?\d{1,2}\.\d{2} (-?\d{1,2}\.\d{2})')
vsp_rename_re = re.compile(r'^Date: (\d{2}/\d{2}/\d{2})$')
# Spectera
spectera_name_re = re.compile(r'Patient Name: ([A-Z- \'\.]+) Subscriber: ([A-Z \'-\.]+) Group Name: .* Claim ID: (\d{14})')
spectera_dos_claim_num_re = re.compile(r'Patient Account: (\w+) SubscriberID: .+ Received Date: (\d{2}/\d{2}/\d{4})')
spectera_payments_re = re.compile(r'SUBTOTAL THIS CLAIM \d{1,3}\.\d{2} \d{1,3}\.\d{2} \d{1,3}\.\d{2} \d{1,3}\.\d{2} (\d{1,3}\.\d{2}) (\d{1,3}\.\d{2}) \d{1,3}\.\d{2}')
spectera_long_name_re = re.compile(r'^([A-Z-\'\.]{2,200}).*')
spectera_rename_re = re.compile(r'^(\d{2}/\d{2}/\d{4}) \$\d{1,3}\.\d{2}$')
# Med Mut
medmut_name_claim_num_re = re.compile(r'PATIENT NAME: ([-A-Z\'\.]+), ([-A-Z\'\.]+) ID NUMBER: \d+ CLAIM NUMBER: (\d{12,14})')
medmut_dos_re = re.compile(r'[N/A\s]*(\d{2}/\d{2}/\d{4}) \d{5} \d{1,3}\.\d{2}')
medmut_payments_re = re.compile(r'CLAIM TOTAL \d{1,3}\.\d{2} \d{1,3}\.\d{2} (\d{1,3})\.\d{2}')
medmut_rename_re = re.compile(r'^(\d{2}-\d{2}-\d{4})$')

Renaming Day Sheets

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/DaySheets/New Sheets')

# Resetting file names
for index, f in enumerate(os.listdir()):
    os.rename(f, str(index))

for f in os.listdir():
    df = pd.read_excel(f)
    same_name_count = 0
    for ft in os.listdir():
        if 'DaySheet_' + str(df['DATE'][0].to_pydatetime().date()) in ft:
            same_name_count += 1
    if same_name_count > 0:
        os.rename(f, 'DaySheet_' + str(df['DATE'][0].to_pydatetime().date()) + ' (' + str(same_name_count) + ')')
    else:
        os.rename(f, 'DaySheet_' + str(df['DATE'][0].to_pydatetime().date()))


Renaming EyeMed Sheets (Website)

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/EyeMed_Sheets/New EyeMed Sheets')

# Resetting file names
for index, f in enumerate(os.listdir()):
    os.rename(f, str(index))

for f in os.listdir():
    with pdfplumber.open(f) as pdf:
        for line in pdf.pages[0].extract_text().split('\n'):
            if eyemed_rename_re.search(line):
                same_name_count = 0
                for ft in os.listdir():
                    if 'EyeMed_' + str(datetime.datetime.strptime(eyemed_rename_re.search(line).group(1), '%m/%d/%y').date()) in ft:
                        same_name_count += 1
                if same_name_count > 0:
                    os.rename(f, 'EyeMed_' + str(datetime.datetime.strptime(eyemed_rename_re.search(line).group(1), '%m/%d/%y').date()) + ' (' + str(same_name_count) + ')')
                else:
                    os.rename(f, 'EyeMed_' + str(datetime.datetime.strptime(eyemed_rename_re.search(line).group(1), '%m/%d/%y').date()))

Renaming EyeMed Sheets (Scanned)

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/EyeMedMisc_Sheets')

# Resetting file names
for index, f in enumerate(os.listdir()):
    os.rename(f, str(index))

for f in os.listdir():
    with pdfplumber.open(f) as pdf:
        for line in pdf.pages[0].extract_text().split('\n'):
            if scanned_eyemed_rename_re.search(line):
                same_name_count = 0
                for ft in os.listdir():
                    if 'Scanned_EyeMed_' + str(datetime.datetime.strptime(scanned_eyemed_rename_re.search(line).group(1).zfill(10), '%m/%d/%Y').date()) in ft:
                        same_name_count += 1
                if same_name_count > 0:
                    os.rename(f, 'Scanned_EyeMed_' + str(datetime.datetime.strptime(scanned_eyemed_rename_re.search(line).group(1).zfill(10), '%m/%d/%Y').date()) + ' (' + str(same_name_count) + ')')
                else:
                    os.rename(f, 'Scanned_EyeMed_' + str(datetime.datetime.strptime(scanned_eyemed_rename_re.search(line).group(1).zfill(10), '%m/%d/%Y').date()))

Renaming VSP Sheets

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/VSP_Sheets/New VSP Sheets')

# Resetting file names
for index, f in enumerate(os.listdir()):
    os.rename(f, str(index))

for f in os.listdir():
    with pdfplumber.open(f) as pdf:
        for line in pdf.pages[0].extract_text().split('\n'):
            if vsp_rename_re.search(line):
                same_name_count = 0
                for ft in os.listdir():
                    if 'VSP_' + str(datetime.datetime.strptime(vsp_rename_re.search(line).group(1), '%m/%d/%y').date()) in ft:
                        same_name_count += 1
                if same_name_count > 0:
                    os.rename(f, 'VSP_' + str(datetime.datetime.strptime(vsp_rename_re.search(line).group(1), '%m/%d/%y').date()) + ' (' + str(same_name_count) + ')')
                else:
                    os.rename(f, 'VSP_' + str(datetime.datetime.strptime(vsp_rename_re.search(line).group(1), '%m/%d/%y').date()))

Renaming Spectera Sheets

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/Spectera_Sheets/New Spectera Sheets')

# Resetting file names
for index, f in enumerate(os.listdir()):
    os.rename(f, str(index))

for f in os.listdir():
    with pdfplumber.open(f) as pdf:
        for line in pdf.pages[0].extract_text().split('\n'):
            if spectera_rename_re.search(line):
                same_name_count = 0
                for ft in os.listdir():
                    if 'Spectera_' + str(datetime.datetime.strptime(spectera_rename_re.search(line).group(1), '%m/%d/%Y').date()) in ft:
                        same_name_count += 1
                if same_name_count > 0:
                    os.rename(f, 'Spectera_' + str(datetime.datetime.strptime(spectera_rename_re.search(line).group(1), '%m/%d/%Y').date()) + ' (' + str(same_name_count) + ')')
                else:
                    os.rename(f, 'Spectera_' + str(datetime.datetime.strptime(spectera_rename_re.search(line).group(1), '%m/%d/%Y').date()))

Renaming MedMut Sheets

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/MedMut_Sheets')

# Resetting file names
for index, f in enumerate(os.listdir()):
    os.rename(f, str(index))

for f in os.listdir():
    with pdfplumber.open(f) as pdf:
        for line in pdf.pages[0].extract_text().split('\n'):
            if medmut_rename_re.search(line):
                same_name_count = 0
                for ft in os.listdir():
                    if 'MedMut_' + str(datetime.datetime.strptime(medmut_rename_re.search(line).group(1).zfill(10), '%m-%d-%Y').date()) in ft:
                        same_name_count += 1
                if same_name_count > 0:
                    os.rename(f, 'MedMut_' + str(datetime.datetime.strptime(medmut_rename_re.search(line).group(1), '%m-%d-%Y').date()) + ' (' + str(same_name_count) + ')')
                else:
                    os.rename(f, 'MedMut_' + str(datetime.datetime.strptime(medmut_rename_re.search(line).group(1), '%m-%d-%Y').date()))

Reading in Day Sheets (EyeMed)

In [ ]:
# Creating the dataframe that will hold all rows of all day sheets
day_eyemed = pd.DataFrame(columns=['Patient Name', 'Date of Service', 'Doctor', 'Expected Insurance Payment', 'Claim Number', 'File Name'])
os.chdir('/content/drive/MyDrive/Insurance/DaySheets/New Sheets')
for f in sorted(os.listdir()):
    
    current_day_sheet = pd.read_excel(f)

    for current_day_sheet_current_row in range(int(current_day_sheet['FIT'][28])):
        
        # Skipping row if it is not an EyeMed patient
        if current_day_sheet['INS CODE'][current_day_sheet_current_row] != 1:
            continue
        
        # Adding optomap to expected insurance payment if necessary
        expected_ins_payment = round(current_day_sheet['EXPECTED'][current_day_sheet_current_row])
        if current_day_sheet['OPTO'][current_day_sheet_current_row] != 28 and str(current_day_sheet['OPTO'][current_day_sheet_current_row]) != 'nan':
            expected_ins_payment += 28 - int(current_day_sheet['OPTO'][current_day_sheet_current_row])
        # Appending the current row to the main dataframe
        day_eyemed = day_eyemed.append({'Patient Name' : current_day_sheet['FIRST NAME'][current_day_sheet_current_row].strip().upper() + ' ' + current_day_sheet['PATIENT LAST NAME'][current_day_sheet_current_row].strip().upper(), 'Date of Service' : current_day_sheet['DATE'][current_day_sheet_current_row].to_pydatetime().date(), 'Doctor' : dr_dict[current_day_sheet['DR'][current_day_sheet_current_row].upper().strip()], 'Expected Insurance Payment' : expected_ins_payment, 'Claim Number' : current_day_sheet['CLAIM NUMBER'][current_day_sheet_current_row], 'File Name' : f}, ignore_index=True)



day_eyemed

Reading in Day Sheets (VSP)

In [ ]:
# Creating the dataframe that will hold all rows of all day sheets
day_vsp = pd.DataFrame(columns=['Patient Name', 'Date of Service', 'Doctor', 'Expected Insurance Payment', 'File Name'])
os.chdir('/content/drive/MyDrive/Insurance/DaySheets/New Sheets')
for f in sorted(os.listdir()):
    
    current_day_sheet = pd.read_excel(f)
    
    for current_day_sheet_current_row in range(int(current_day_sheet['FIT'][28])):
        
        # Skipping row if it is not a VSP patient
        if current_day_sheet['INS CODE'][current_day_sheet_current_row] != 2:
            continue
        
        # Adding optomap to expected insurance payment if necessary
        expected_ins_payment = round(current_day_sheet['EXPECTED'][current_day_sheet_current_row])
        if current_day_sheet['OPTO'][current_day_sheet_current_row] != 28 and str(current_day_sheet['OPTO'][current_day_sheet_current_row]) != 'nan':
            expected_ins_payment += 28 - int(current_day_sheet['OPTO'][current_day_sheet_current_row])
            print(f)
        # Appending the current row to the main dataframe
        day_vsp = day_vsp.append({'Patient Name' : current_day_sheet['FIRST NAME'][current_day_sheet_current_row].strip().upper() + ' ' + current_day_sheet['PATIENT LAST NAME'][current_day_sheet_current_row].strip().upper(), 'Date of Service' : current_day_sheet['DATE'][current_day_sheet_current_row].to_pydatetime().date(), 'Doctor' : dr_dict[current_day_sheet['DR'][current_day_sheet_current_row].upper().strip()], 'Expected Insurance Payment' : expected_ins_payment, 'File Name' : f}, ignore_index=True)

day_vsp

Reading in Day Sheets (Spectera)

In [ ]:
# Creating the dataframe that will hold all rows of all day sheets
day_spectera = pd.DataFrame(columns=['Patient Name', 'Date of Service', 'Doctor', 'Expected Insurance Payment', 'Claim Number', 'File Name'])
os.chdir('/content/drive/MyDrive/Insurance/DaySheets/New Sheets')
for f in sorted(os.listdir()):
    
    current_day_sheet = pd.read_excel(f)
    
    for current_day_sheet_current_row in range(int(current_day_sheet['FIT'][28])):
        
        # Skipping row if it is not a Spectera patient
        if current_day_sheet['INS CODE'][current_day_sheet_current_row] != 3:
            continue
        
        # Adding optomap to expected insurance payment if necessary
        expected_ins_payment = round(current_day_sheet['EXPECTED'][current_day_sheet_current_row])
        if current_day_sheet['OPTO'][current_day_sheet_current_row] != 28 and str(current_day_sheet['OPTO'][current_day_sheet_current_row]) != 'nan':
            expected_ins_payment += 28 - int(current_day_sheet['OPTO'][current_day_sheet_current_row])
            print(f)
        # Appending the current row to the main dataframe
        day_spectera = day_spectera.append({'Patient Name' : current_day_sheet['FIRST NAME'][current_day_sheet_current_row].strip().upper() + ' ' + current_day_sheet['PATIENT LAST NAME'][current_day_sheet_current_row].strip().upper(), 'Date of Service' : current_day_sheet['DATE'][current_day_sheet_current_row].to_pydatetime().date(), 'Doctor' : dr_dict[current_day_sheet['DR'][current_day_sheet_current_row].upper().strip()], 'Expected Insurance Payment' : expected_ins_payment, 'Claim Number' : str(current_day_sheet['CLAIM NUMBER'][current_day_sheet_current_row]).upper(), 'File Name' : f}, ignore_index=True)



day_spectera

Reading in Insurance Sheets (EyeMed)

In [ ]:
# Sheets off of Website
eyemed_ins_df = pd.DataFrame(columns=['Patient Name', 'Date of Service', 'Insurance Payment', 'Claim Number', 'File Name'])

num_failures = 0
num_successes = 0
successfully_appended_df = True # Setting it to true for the beginning of the loops

os.chdir('/content/drive/MyDrive/Insurance/EyeMed_Sheets/New EyeMed Sheets')
for eyemed_pdf in os.listdir():
    with pdfplumber.open(eyemed_pdf) as current_eyemed_pdf:
        name_found = False
        claim_num_found = False
        dos_found = False
        payment_found = False
        profile = []
        for page in current_eyemed_pdf.pages:

            for line in page.extract_text().split('\n'):
                
                if eyemed_claim_num_re.search(line):

                    if not successfully_appended_df:
                        num_failures += 1
                    else:
                        num_successes += 1
                    
                    profile = []
                    profile.append(int(eyemed_claim_num_re.search(line).group(1)))
                    profile.append(str(eyemed_pdf))
                    
                    name_found = False
                    claim_num_found = False
                    dos_found = False
                    payment_found = False
                    successfully_appended_df = False
                    
                    claim_num_found = True

                elif eyemed_name_re.search(line) and claim_num_found:
                    profile.append(str(eyemed_name_re.search(line).group(2)) + ' ' + str(eyemed_name_re.search(line).group(1)))
                    name_found = True
                elif eyemed_dos_re.search(line) and not dos_found and name_found and claim_num_found:
                    profile.append(datetime.datetime.strptime(str(eyemed_dos_re.search(line).group(1)).strip(), '%m/%d/%y').date())
                    dos_found = True
                elif eyemed_payments_re.search(line) and name_found and claim_num_found and dos_found:
                    profile.append(round(float(eyemed_payments_re.search(line).group(4))))
                    payment_found = True

                if len(profile) == 5:
                    eyemed_ins_df = eyemed_ins_df.append({'Patient Name' : profile[2], 'Claim Number' : profile[0], 'Date of Service' : profile[3], 'Insurance Payment' : profile[4], 'File Name' : profile[1]}, ignore_index=True)
                    profile = []
                    name_found = False
                    claim_num_found = False
                    dos_found = False
                    payment_found = False

                    successfully_appended_df = True

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Sheets that were scanned
os.chdir('/content/drive/MyDrive/Insurance/EyeMedMisc_Sheets')

successfully_appended_df = True # Setting it to true for the beginning of the loops

for scanned_eyemed_pdf_filename in os.listdir():
    with pdfplumber.open(scanned_eyemed_pdf_filename) as open_eyemed_pdf:
        name_claim_num_found = False
        dos_found = False
        payment_found = False
        profile = []
        for page in open_eyemed_pdf.pages:

            for line in page.extract_text().split('\n'):
                
                if scanned_eyemed_name_claim_num_re.search(line):
                    if not successfully_appended_df:
                        num_failures += 1
                    else:
                        num_successes += 1
                    
                    profile = []
                    profile.append(scanned_eyemed_name_claim_num_re.search(line).group(3) + ' ' + scanned_eyemed_name_claim_num_re.search(line).group(2))
                    profile.append(scanned_eyemed_name_claim_num_re.search(line).group(1))
                    profile.append(scanned_eyemed_pdf_filename)
                    name_claim_num_found = False
                    dos_found = False
                    payment_found = False
                    successfully_appended_df = False

                    name_claim_num_found = True
                
                elif scanned_eyemed_dos_re.search(line) and not dos_found and name_claim_num_found:
                    profile.append(datetime.datetime.strptime(scanned_eyemed_dos_re.search(line).group(1).strip().zfill(10), '%m/%d/%Y').date())
                    dos_found = True
                        
                elif scanned_eyemed_payments_re.search(line) and dos_found and name_claim_num_found:
                    profile.append(scanned_eyemed_payments_re.search(line).group(1))
                if len(profile) == 5:
                    eyemed_ins_df = eyemed_ins_df.append({'Patient Name' : profile[0], 'Claim Number' : profile[1], 'Date of Service' : profile[3], 'Insurance Payment' : profile[4], 'File Name' : profile[2]}, ignore_index=True)
                    profile = []
                    name_found = False
                    claim_num_found = False
                    dos_found = False
                    payment_found = False
                    successfully_appended_df = True



In [ ]:
eyemed_ins_df

Reading in Insurance Sheets (VSP)

In [ ]:
vsp_ins_df = pd.DataFrame(columns=['Patient Name', 'Date of Service', 'Insurance Payment', 'Claim Number', 'File Name'])

os.chdir('/content/drive/MyDrive/Insurance/VSP_Sheets/New VSP Sheets')

num_failures = 0
num_successes = 0
successfully_appended_df = True # Setting it to true for the beginning of the loops

for vsp_ins_pdf_filename in os.listdir():
    with pdfplumber.open(vsp_ins_pdf_filename) as open_vsp_pdf:
        name_claim_num_found = False
        dos_found = False
        payment_found = False
        profile = []

        for page in open_vsp_pdf.pages:
            for line in page.extract_text().split('\n'):
            
                if vsp_name_claim_num_re.search(line):
                    if not successfully_appended_df:
                        num_failures += 1
                    else:
                        num_successes += 1
                    profile = []
                    profile.append(str(vsp_name_claim_num_re.search(line).group(2)) + ' ' + str(vsp_name_claim_num_re.search(line).group(1)))
                    profile.append(int(vsp_name_claim_num_re.search(line).group(3))) #cn
                    profile.append(str(vsp_ins_pdf_filename))

                    name_claim_num_found = False
                    dos_found = False
                    payment_found = False
                    
                    name_claim_num_found  = True
                elif vsp_dos_re.search(line) and not dos_found and name_claim_num_found:
                    profile.append(datetime.datetime.strptime(str(vsp_dos_re.search(line).group(1)).strip().zfill(8), '%m/%d/%y').date())
                    dos_found = True
                elif vsp_payments_re.search(line) and dos_found and name_claim_num_found:
                    profile.append(round(float(vsp_payments_re.search(line).group(1))))
                    payment_found = True
                if len(profile) == 5:
                    vsp_ins_df = vsp_ins_df.append({'Patient Name' : profile[0], 'Claim Number' : profile[1], 'Date of Service' : profile[3], 'Insurance Payment' : profile[4], 'File Name' : profile[2]}, ignore_index=True)
                    profile = []
                    name_claim_num_found = False
                    dos_found = False
                    payment_found = False
                    successfully_appended_df = True

In [ ]:
vsp_ins_df

Reading in Insurance Sheets (Spectera)

In [ ]:
spectera_ins_df = pd.DataFrame(columns=['Patient Name', 'Date of Service', 'Insurance Payment', 'Claim Number', 'File Name'])

os.chdir('/content/drive/MyDrive/Insurance/Spectera_Sheets/New Spectera Sheets')

num_failures = 0
num_successes = 0
successfully_appended_df = True # Setting it to true for the beginning of the loops

for spectera_pdf_filename in os.listdir():
    with pdfplumber.open(spectera_pdf_filename) as open_spectera_pdf:
        name_found = False
        dos_claim_num_found = False
        payment_found = False
        profile = []
        for page in open_spectera_pdf.pages[2:]:
            line_list = page.extract_text().split('\n')
            for index, line in enumerate(line_list):
                if spectera_name_re.search(line):

                    name_found = False
                    dos_claim_num_found = False
                    payment_found = False
                    successfully_appended_df = False
                    profile = []
                    
                    if not spectera_dos_claim_num_re.search(line_list[index + 1]) and spectera_long_name_re.search(line_list[index + 1]):
                        on_left_side = False
                        for dictionary in page.extract_words(keep_blank_chars=False):
                            if spectera_long_name_re.search(line_list[index + 1]).group(1).strip() in dictionary['text'] and dictionary['x0'] < 150:
                                profile.append(str(spectera_name_re.search(line).group(1).strip() + ' ' + spectera_long_name_re.search(line_list[index + 1]).group(1).strip()))
                                on_left_side = True
                        if not on_left_side:
                            profile.append(str(spectera_name_re.search(line).group(1).strip()))
                    else:
                        profile.append(str(spectera_name_re.search(line).group(1).strip()))
                    
                    profile.append(str(spectera_pdf_filename))

                    name_found = True
                    
                elif spectera_dos_claim_num_re.search(line) and name_found:
                    profile.append(datetime.datetime.strptime(str(spectera_dos_claim_num_re.search(line).group(2)), '%m/%d/%Y')) # dos
                    profile.append(str(spectera_dos_claim_num_re.search(line).group(1))) # cn
                    dos_claim_num_found = True
                elif spectera_payments_re.search(line) and dos_claim_num_found and name_found:
                    profile.append(round(float(spectera_payments_re.search(line).group(2))))
                    payment_found = True
                if len(profile) == 5:
                    spectera_ins_df = spectera_ins_df.append({'Patient Name' : profile[0], 'Claim Number' : profile[3], 'Date of Service' : profile[2], 'Insurance Payment' : profile[4], 'File Name' : profile[1]}, ignore_index=True)
                    profile = []
                    name_found = False
                    dos_claim_num_found = False
                    payment_found = False
                    successfully_appended_df = True


In [ ]:
spectera_ins_df

Reading in Insurance Sheets (MedMut)

In [ ]:
medmut_ins_df = pd.DataFrame(columns=['Patient Name', 'Date of Service', 'Insurance Payment', 'Claim Number', 'File Name'])

os.chdir('/content/drive/MyDrive/Insurance/MedMut_Sheets')

num_failures = 0
num_successes = 0
successfully_appended_df = True # Setting it to true for the beginning of the loops

for medmut_ins_pdf_filename in os.listdir():
    with pdfplumber.open(medmut_ins_pdf_filename) as open_medmut_pdf:
        name_claim_num_found = False
        dos_found = False
        payment_found = False
        profile = []

        for page in open_medmut_pdf.pages:
            for line in page.extract_text().split('\n'):
            
                if medmut_name_claim_num_re.search(line):
                    if not successfully_appended_df:
                        num_failures += 1
                    else:
                        num_successes += 1
                    profile = []
                    profile.append(str(medmut_name_claim_num_re.search(line).group(2)) + ' ' + str(medmut_name_claim_num_re.search(line).group(1)))
                    profile.append(int(medmut_name_claim_num_re.search(line).group(3))) #cn
                    profile.append(str(medmut_ins_pdf_filename))

                    name_claim_num_found = False
                    dos_found = False
                    payment_found = False
                    
                    name_claim_num_found  = True
                elif medmut_dos_re.search(line) and not dos_found and name_claim_num_found:
                    profile.append(datetime.datetime.strptime(str(medmut_dos_re.search(line).group(1)).strip(), '%m/%d/%Y').date())
                    dos_found = True
                elif medmut_payments_re.search(line) and dos_found and name_claim_num_found:
                    profile.append(round(float(medmut_payments_re.search(line).group(1))))
                    payment_found = True
                if len(profile) == 5:
                    medmut_ins_df = medmut_ins_df.append({'Patient Name' : profile[0], 'Claim Number' : profile[1], 'Date of Service' : profile[3], 'Insurance Payment' : profile[4], 'File Name' : profile[2]}, ignore_index=True)
                    profile = []
                    name_claim_num_found = False
                    dos_found = False
                    payment_found = False
                    successfully_appended_df = True

In [ ]:
medmut_ins_df

Finding Matches (EyeMed)

In [ ]:
eyemed_matches_df = pd.DataFrame(columns=['Insurance Used', 'Doctor', 'Patient Name (Day Sheet)', 'Patient Name (Insurance)', 'Date of Service (Day Sheet)', 'Date of Service (Insurance)', 'Expected Insurance Payment (Day Sheet)', 'Insurance Payment (Insurance)', 'Claim Number (Day Sheet)', 'Claim Number (Insurance)', 'File Name (Day Sheet)', 'File Name (Insurance)'])
eyemed_discs_df = pd.DataFrame(columns=['Resolved?', 'Insurance Payment Not Found', 'Payments Don\'t Match', 'Insurance Used', 'Doctor', 'Patient Name (Day Sheet)', 'Patient Name (Insurance)', 'Date of Service (Day Sheet)', 'Date of Service (Insurance)', 'Expected Insurance Payment (Day Sheet)', 'Insurance Payment (Insurance)', 'Claim Number (Day Sheet)', 'Claim Number (Insurance)', 'File Name (Day Sheet)', 'File Name (Insurance)'])

match_found = True # Set to true so the loop will begin properly
for index, current_day_eyemed_row in day_eyemed.iterrows():
    for indextwo, current_eyemed_ins_df_row in eyemed_ins_df.iterrows():
        # If a match is found
        if current_day_eyemed_row['Date of Service'] == current_eyemed_ins_df_row['Date of Service'] and current_day_eyemed_row['Claim Number'] == current_eyemed_ins_df_row['Claim Number']:
            match_found = True
            if abs(current_day_eyemed_row['Expected Insurance Payment'] - current_eyemed_ins_df_row['Insurance Payment']) <= 1:
                eyemed_matches_df = eyemed_matches_df.append({'Insurance Used' : 'Eye Med', 'Doctor' : current_day_eyemed_row['Doctor'],  'Patient Name (Day Sheet)' : current_day_eyemed_row['Patient Name'], 'Patient Name (Insurance)' : current_eyemed_ins_df_row['Patient Name'], 'Date of Service (Day Sheet)' : current_day_eyemed_row['Date of Service'], 'Date of Service (Insurance)' : current_eyemed_ins_df_row['Date of Service'], 'Expected Insurance Payment (Day Sheet)' : current_day_eyemed_row['Expected Insurance Payment'], 'Insurance Payment (Insurance)' : current_eyemed_ins_df_row['Insurance Payment'], 'Claim Number (Day Sheet)' : current_day_eyemed_row['Claim Number'], 'Claim Number (Insurance)' : current_eyemed_ins_df_row['Claim Number'], 'File Name (Day Sheet)' : current_day_eyemed_row['File Name'], 'File Name (Insurance)' : current_eyemed_ins_df_row['File Name']}, ignore_index=True)
            else:
                eyemed_discs_df = eyemed_discs_df.append({'Payments Don\'t Match' : 'X', 'Insurance Used' : 'Eye Med', 'Doctor' : current_day_eyemed_row['Doctor'],  'Patient Name (Day Sheet)' : current_day_eyemed_row['Patient Name'], 'Patient Name (Insurance)' : current_eyemed_ins_df_row['Patient Name'], 'Date of Service (Day Sheet)' : current_day_eyemed_row['Date of Service'], 'Date of Service (Insurance)' : current_eyemed_ins_df_row['Date of Service'], 'Expected Insurance Payment (Day Sheet)' : current_day_eyemed_row['Expected Insurance Payment'], 'Insurance Payment (Insurance)' : current_eyemed_ins_df_row['Insurance Payment'], 'Claim Number (Day Sheet)' : current_day_eyemed_row['Claim Number'], 'Claim Number (Insurance)' : current_eyemed_ins_df_row['Claim Number'], 'File Name (Day Sheet)' : current_day_eyemed_row['File Name'], 'File Name (Insurance)' : current_eyemed_ins_df_row['File Name']}, ignore_index=True)
            break

    if not match_found:
        eyemed_discs_df = eyemed_discs_df.append({'Insurance Payment Not Found' : 'X', 'Insurance Used' : 'Eye Med', 'Doctor' : current_day_eyemed_row['Doctor'],  'Patient Name (Day Sheet)' : current_day_eyemed_row['Patient Name'], 'Date of Service (Day Sheet)' : current_day_eyemed_row['Date of Service'], 'Expected Insurance Payment (Day Sheet)' : current_day_eyemed_row['Expected Insurance Payment'], 'Claim Number (Day Sheet)' : current_day_eyemed_row['Claim Number'], 'File Name (Day Sheet)' : current_day_eyemed_row['File Name']}, ignore_index=True)
    match_found = False



Finding Matches (VSP)

In [ ]:
vsp_matches_df = pd.DataFrame(columns=['Insurance Used', 'Doctor', 'Patient Name (Day Sheet)', 'Patient Name (Insurance)', 'Date of Service (Day Sheet)', 'Date of Service (Insurance)', 'Expected Insurance Payment (Day Sheet)', 'Insurance Payment (Insurance)', 'File Name (Day Sheet)', 'File Name (Insurance)'])
vsp_discs_df = pd.DataFrame(columns=['Resolved?', 'Insurance Payment Not Found', 'Payments Don\'t Match', 'Insurance Used', 'Doctor', 'Patient Name (Day Sheet)', 'Patient Name (Insurance)', 'Date of Service (Day Sheet)', 'Date of Service (Insurance)', 'Expected Insurance Payment (Day Sheet)', 'Insurance Payment (Insurance)', 'File Name (Day Sheet)', 'File Name (Insurance)'])

match_found = True # Set to true so the loop will begin properly
for index, current_day_vsp_row in day_vsp.iterrows():
    for indextwo, current_vsp_ins_df_row in vsp_ins_df.iterrows():
        # If a match is found
        if match_or_not_vsp(current_day_vsp_row['Patient Name'], current_vsp_ins_df_row['Patient Name'], current_day_vsp_row['Date of Service'], current_vsp_ins_df_row['Date of Service']):
            match_found = True
            if abs(current_day_vsp_row['Expected Insurance Payment'] - current_vsp_ins_df_row['Insurance Payment']) <= 1:
                vsp_matches_df = vsp_matches_df.append({'Insurance Used' : 'VSP', 'Doctor' : current_day_vsp_row['Doctor'],  'Patient Name (Day Sheet)' : current_day_vsp_row['Patient Name'], 'Patient Name (Insurance)' : current_vsp_ins_df_row['Patient Name'], 'Date of Service (Day Sheet)' : current_day_vsp_row['Date of Service'], 'Date of Service (Insurance)' : current_vsp_ins_df_row['Date of Service'], 'Expected Insurance Payment (Day Sheet)' : current_day_vsp_row['Expected Insurance Payment'], 'Insurance Payment (Insurance)' : current_vsp_ins_df_row['Insurance Payment'], 'File Name (Day Sheet)' : current_day_vsp_row['File Name'], 'File Name (Insurance)' : current_vsp_ins_df_row['File Name']}, ignore_index=True)
            else:
                vsp_discs_df = vsp_discs_df.append({'Payments Don\'t Match' : 'X', 'Insurance Used' : 'VSP', 'Doctor' : current_day_vsp_row['Doctor'],  'Patient Name (Day Sheet)' : current_day_vsp_row['Patient Name'], 'Patient Name (Insurance)' : current_vsp_ins_df_row['Patient Name'], 'Date of Service (Day Sheet)' : current_day_vsp_row['Date of Service'], 'Date of Service (Insurance)' : current_vsp_ins_df_row['Date of Service'], 'Expected Insurance Payment (Day Sheet)' : current_day_vsp_row['Expected Insurance Payment'], 'Insurance Payment (Insurance)' : current_vsp_ins_df_row['Insurance Payment'], 'File Name (Day Sheet)' : current_day_vsp_row['File Name'], 'File Name (Insurance)' : current_vsp_ins_df_row['File Name']}, ignore_index=True)
            break

    if not match_found:
        vsp_discs_df = vsp_discs_df.append({'Insurance Payment Not Found' : 'X', 'Insurance Used' : 'VSP', 'Doctor' : current_day_vsp_row['Doctor'],  'Patient Name (Day Sheet)' : current_day_vsp_row['Patient Name'], 'Date of Service (Day Sheet)' : current_day_vsp_row['Date of Service'], 'Expected Insurance Payment (Day Sheet)' : current_day_vsp_row['Expected Insurance Payment'], 'File Name (Day Sheet)' : current_day_vsp_row['File Name']}, ignore_index=True)
    match_found = False



Finding Matches (Spectera)

In [ ]:
spectera_matches_df = pd.DataFrame(columns=['Insurance Used', 'Doctor', 'Patient Name (Day Sheet)', 'Patient Name (Insurance)', 'Date of Service (Day Sheet)', 'Date of Service (Insurance)', 'Expected Insurance Payment (Day Sheet)', 'Insurance Payment (Insurance)', 'Claim Number (Day Sheet)', 'Claim Number (Insurance)', 'File Name (Day Sheet)', 'File Name (Insurance)'])
spectera_discs_df = pd.DataFrame(columns=['Resolved?', 'Insurance Payment Not Found', 'Payments Don\'t Match', 'Insurance Used', 'Doctor', 'Patient Name (Day Sheet)', 'Patient Name (Insurance)', 'Date of Service (Day Sheet)', 'Date of Service (Insurance)', 'Expected Insurance Payment (Day Sheet)', 'Insurance Payment (Insurance)', 'Claim Number (Day Sheet)', 'Claim Number (Insurance)', 'File Name (Day Sheet)', 'File Name (Insurance)'])

match_found = True # Set to true so the loop will begin properly
for index, current_day_spectera_row in day_spectera.iterrows():
    for indextwo, current_spectera_ins_df_row in spectera_ins_df.iterrows():
        # If a match is found
        if current_day_spectera_row['Date of Service'] == current_spectera_ins_df_row['Date of Service'] and current_day_spectera_row['Claim Number'] == current_spectera_ins_df_row['Claim Number']:
            match_found = True
            if abs(current_day_spectera_row['Expected Insurance Payment'] - current_spectera_ins_df_row['Insurance Payment']) <= 1:
                spectera_matches_df = spectera_matches_df.append({'Insurance Used' : 'Eye Med', 'Doctor' : current_day_spectera_row['Doctor'],  'Patient Name (Day Sheet)' : current_day_spectera_row['Patient Name'], 'Patient Name (Insurance)' : current_spectera_ins_df_row['Patient Name'], 'Date of Service (Day Sheet)' : current_day_spectera_row['Date of Service'], 'Date of Service (Insurance)' : current_spectera_ins_df_row['Date of Service'], 'Expected Insurance Payment (Day Sheet)' : current_day_spectera_row['Expected Insurance Payment'], 'Insurance Payment (Insurance)' : current_spectera_ins_df_row['Insurance Payment'], 'Claim Number (Day Sheet)' : current_day_spectera_row['Claim Number'], 'Claim Number (Insurance)' : current_spectera_ins_df_row['Claim Number'], 'File Name (Day Sheet)' : current_day_spectera_row['File Name'], 'File Name (Insurance)' : current_spectera_ins_df_row['File Name']}, ignore_index=True)
            else:
                spectera_discs_df = spectera_discs_df.append({'Payments Don\'t Match' : 'X', 'Insurance Used' : 'Eye Med', 'Doctor' : current_day_spectera_row['Doctor'],  'Patient Name (Day Sheet)' : current_day_spectera_row['Patient Name'], 'Patient Name (Insurance)' : current_spectera_ins_df_row['Patient Name'], 'Date of Service (Day Sheet)' : current_day_spectera_row['Date of Service'], 'Date of Service (Insurance)' : current_spectera_ins_df_row['Date of Service'], 'Expected Insurance Payment (Day Sheet)' : current_day_spectera_row['Expected Insurance Payment'], 'Insurance Payment (Insurance)' : current_spectera_ins_df_row['Insurance Payment'], 'Claim Number (Day Sheet)' : current_day_spectera_row['Claim Number'], 'Claim Number (Insurance)' : current_spectera_ins_df_row['Claim Number'], 'File Name (Day Sheet)' : current_day_spectera_row['File Name'], 'File Name (Insurance)' : current_spectera_ins_df_row['File Name']}, ignore_index=True)
            break

    if not match_found:
        spectera_discs_df = spectera_discs_df.append({'Insurance Payment Not Found' : 'X', 'Insurance Used' : 'Eye Med', 'Doctor' : current_day_spectera_row['Doctor'],  'Patient Name (Day Sheet)' : current_day_spectera_row['Patient Name'], 'Date of Service (Day Sheet)' : current_day_spectera_row['Date of Service'], 'Expected Insurance Payment (Day Sheet)' : current_day_spectera_row['Expected Insurance Payment'], 'Claim Number (Day Sheet)' : current_day_spectera_row['Claim Number'], 'File Name (Day Sheet)' : current_day_spectera_row['File Name']}, ignore_index=True)
    match_found = False


In [ ]:
spectera_matches_df

Exporting EyeMed Excel

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/Disc Sheets/')

date_series = sorted(eyemed_matches_df['Date of Service (Day Sheet)'].append(eyemed_discs_df['Date of Service (Day Sheet)']))
name_excel_add_str = '_' + str(date_series[0]) + '_through_' + str(date_series[-1])

eyemed_matches_df.to_excel('EyeMed_Matches' + name_excel_add_str + '.xlsx')
eyemed_discs_df.to_excel('EyeMed_Discs' + name_excel_add_str + '.xlsx')

Exporting VSP Excel

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/Disc Sheets/')

date_series = sorted(vsp_matches_df['Date of Service (Day Sheet)'].append(vsp_discs_df['Date of Service (Day Sheet)']))
name_excel_add_str = '_' + str(date_series[0]) + '_through_' + str(date_series[-1])

vsp_matches_df.to_excel('vsp_Matches' + name_excel_add_str + '.xlsx')
vsp_discs_df.to_excel('vsp_Discs' + name_excel_add_str + '.xlsx')

Exporting Spectera Excel

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/Disc Sheets/')

date_series = sorted(spectera_matches_df['Date of Service (Day Sheet)'].append(spectera_discs_df['Date of Service (Day Sheet)']))
name_excel_add_str = '_' + str(date_series[0]) + '_through_' + str(date_series[-1])

spectera_matches_df.to_excel('spectera_Matches' + name_excel_add_str + '.xlsx')
spectera_discs_df.to_excel('spectera_Discs' + name_excel_add_str + '.xlsx')

Exporting MedMut Excel

In [ ]:
os.chdir('/content/drive/MyDrive/Insurance/Disc Sheets/')

date_series = sorted(medmut_matches_df['Date of Service (Day Sheet)'].append(medmut_discs_df['Date of Service (Day Sheet)']))
name_excel_add_str = '_' + str(date_series[0]) + '_through_' + str(date_series[-1])

medmut_matches_df.to_excel('medmut_Matches' + name_excel_add_str + '.xlsx')
medmut_discs_df.to_excel('medmut_Discs' + name_excel_add_str + '.xlsx')